# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [64]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

from jupyter_core.migrate import regex


def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def get_area(team):
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.at[each, 'Metropolitan area']


nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
nhl_cities = nhl_cities.drop(['—', ''], axis=0)
nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nhl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
nhl_out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
nhl_out_df['Population (2016 est.)[8]'] = pd.to_numeric(nhl_out_df['Population (2016 est.)[8]'])

def nhl_correlation():
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = nhl_out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = nhl_out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])


def get_nhl_data():
    return nhl_out_df

In [65]:
get_nhl_data()

,Ratio,Population (2016 est.)[8]
Boston,0.714286,4794447
Buffalo,0.357143,1132804
Calgary,0.513889,1392609
Chicago,0.458333,9512999
Columbus,0.600000,2041520
Dallas–Fort Worth,0.567568,7233323
Denver,0.589041,2853077
Detroit,0.434783,4297617
Edmonton,0.473684,1321426
Las Vegas,0.680000,2155664


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [62]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def get_area(team):
    for each in list(nba_cities.index.values):
        if team in each:
            return nba_cities.at[each, 'Metropolitan area']


nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NBA'] = cities['NBA'].apply(lambda x: clear_data(x))
nba_cities = cities[['Metropolitan area', 'NBA']].set_index('NBA')
nba_cities = nba_cities.drop(['—', ''], axis=0)
nba_df = nba_df[nba_df['year'] == 2018] # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
# nba_df['team'] = nba_df['team'].apply(lambda x: clear_nba_data(x))
nba_df['team'] = nba_df['team'].str.replace(r'\*?\s*\(\d+\)$',"", regex=True)
nba_df['area'] = nba_df['team'].apply(lambda x: x.split(" ")[-1])
nba_df['area'] = nba_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nba_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
nba_out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
nba_out_df['Population (2016 est.)[8]'] = pd.to_numeric(nba_out_df['Population (2016 est.)[8]'])


def nba_correlation():
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = nba_out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = nba_out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])


def get_nba_data():
    return nba_out_df

In [63]:
get_nba_data()

,Ratio,Population (2016 est.)[8]
Atlanta,0.292683,5789700
Boston,0.670732,4794447
Charlotte,0.439024,2474314
Chicago,0.329268,9512999
Cleveland,0.609756,2055612
Dallas–Fort Worth,0.292683,7233323
Denver,0.560976,2853077
Detroit,0.475610,4297617
Houston,0.792683,6772470
Indianapolis,0.585366,2004230


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [66]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def get_area(team):
    for each in list(mlb_cities.index.values):
        if team in each:
            return mlb_cities.at[each, 'Metropolitan area']

def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x))
mlb_cities = cities[['Metropolitan area', 'MLB']].set_index('MLB')
mlb_cities = mlb_cities.drop(['—', ''], axis=0)
mlb_df = mlb_df[mlb_df['year'] == 2018]  # get only 2018 stats no need of dropping rows
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
mlb_df['area'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])
mlb_df['area'] = mlb_df['area'].apply(lambda x: get_area(x))
mlb_df.at[0, 'area'] = 'Boston'
out = []
for group, frame in mlb_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
mlb_out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
mlb_out_df['Population (2016 est.)[8]'] = pd.to_numeric(mlb_out_df['Population (2016 est.)[8]'])


def mlb_correlation():
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = mlb_out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = mlb_out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_mlb_data():
    return mlb_out_df

In [67]:
get_mlb_data()

,Ratio,Population (2016 est.)[8]
Atlanta,0.555556,5789700
Baltimore,0.290123,2798886
Boston,0.666667,4794447
Chicago,0.483077,9512999
Cincinnati,0.413580,2165139
Cleveland,0.561728,2055612
Dallas–Fort Worth,0.413580,7233323
Denver,0.558282,2853077
Detroit,0.395062,4297617
Houston,0.635802,6772470


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [68]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def clear_nba_data(string1):
    if re.search(r'\*|\+', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\*|\+', string1).group(), '')


def get_area(team):
    for each in list(nfl_cities.index.values):
        if team in each:
            return nfl_cities.at[each, 'Metropolitan area']



cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NFL'] = cities['NFL'].apply(lambda x: clear_data(x))
nfl_cities = cities[['Metropolitan area', 'NFL']].set_index('NFL')
nfl_cities = nfl_cities.drop(['—', ''], axis=0)
nfl_df = pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[nfl_df['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35])  # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
nfl_df['team'] = nfl_df['team'].apply(lambda x: clear_nba_data(x))
nfl_df['area'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1])
nfl_df['area'] = nfl_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nfl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
nfl_out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
nfl_out_df['Population (2016 est.)[8]'] = pd.to_numeric(nfl_out_df['Population (2016 est.)[8]'])

def nfl_correlation():
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = nfl_out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = nfl_out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_nfl_data():
    return nfl_out_df

In [69]:
get_nfl_data()

,Ratio,Population (2016 est.)[8]
Atlanta,0.437500,5789700
Baltimore,0.625000,2798886
Boston,0.687500,4794447
Buffalo,0.375000,1132804
Charlotte,0.437500,2474314
Chicago,0.750000,9512999
Cincinnati,0.375000,2165139
Cleveland,0.466667,2055612
Dallas–Fort Worth,0.625000,7233323
Denver,0.375000,2853077


In [56]:
MLB = get_mlb_data().drop('Population (2016 est.)[8]', axis=1)

NHL = get_nhl_data().drop('Population (2016 est.)[8]', axis=1)
NBA = get_nba_data().drop('Population (2016 est.)[8]', axis=1)
NFL = get_nfl_data().drop('Population (2016 est.)[8]', axis=1)
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
leagues = {'NFL': NFL,
            'NBA': NBA,
            'NHL': NHL,
            'MLB': MLB}
sports = ['NFL', 'NBA', 'NHL', 'MLB']
def calculate_p_values(leag):
    p_values = pd.DataFrame(columns=leag.keys(), index=leag.keys())
    for (league_name1, league_df1) in leag.items():
        for (league_name2, league_df2) in leag.items():
            merged_league = pd.merge(league_df1, league_df2, how="inner", left_index=True, right_index=True)
            p_values.loc[league_name1, league_name2] = stats.ttest_rel(merged_league["Ratio_x"], merged_league["Ratio_y"])[1]

    return p_values
p_values_dict = calculate_p_values(leagues)
p_values_dict

C:\Program Files\JetBrains\PyCharm 2024.3.2\plugins\python-ce\helpers\pydev\_pydevd_bundle\pydevd_vars.py:725: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kind = df.dtypes[0].kind
C:\Program Files\JetBrains\PyCharm 2024.3.2\plugins\python-ce\helpers\pydev\_pydevd_bundle\pydevd_vars.py:725: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kind = df.dtypes[0].kind


KeyboardInterrupt: 

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [70]:
import pandas as pd
import scipy.stats as stats


MLB = get_mlb_data().drop('Population (2016 est.)[8]', axis=1)
NHL = get_nhl_data().drop('Population (2016 est.)[8]', axis=1)
NBA = get_nba_data().drop('Population (2016 est.)[8]', axis=1)
NFL = get_nfl_data().drop('Population (2016 est.)[8]', axis=1)
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
leagues = {'NFL': NFL,
            'NBA': NBA,
            'NHL': NHL,
            'MLB': MLB}
sports = ['NFL', 'NBA', 'NHL', 'MLB']

def calculate_p_values(leag):
    p_values = pd.DataFrame(columns=leag.keys(), index=leag.keys())
    for (league_name1, league_df1) in leag.items():
        for (league_name2, league_df2) in leag.items():
            merged_league = pd.merge(league_df1, league_df2, how="inner", left_index=True, right_index=True)
            p_values.loc[league_name1, league_name2] = stats.ttest_rel(merged_league["Ratio_x"], merged_league["Ratio_y"])[1]

    return p_values

def sports_team_performance():
    p_values_dict = calculate_p_values(leagues)
    p_values = pd.DataFrame(p_values_dict).astype("float")
    #assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    #assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [71]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
